In [1]:
!pip install yfinance requests beautifulsoup4

In [2]:
ANTHROPIC_API_KEY = ""  # Replace with your Anthropic API key

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import ast
import json


def get_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article_text = ' '.join([p.get_text() for p in soup.find_all('p')])
        return article_text
    except:
        return "Error retrieving article text."

def get_stock_data(ticker, years):
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=years*365)

    stock = yf.Ticker(ticker)

    # Retrieve historical price data
    hist_data = stock.history(start=start_date, end=end_date)

    # Retrieve balance sheet
    balance_sheet = stock.balance_sheet

    # Retrieve financial statements
    financials = stock.financials

    # Retrieve news articles
    news = stock.news

    return hist_data, balance_sheet, financials, news

def get_claude_comps_analysis(ticker, hist_data, balance_sheet, financials, news):
    system_prompt = f"You are a seasoned hedge fund manager with a proven track record of generating high returns with low risk. Your task is to thoroughly analyze the provided data for {ticker}, which includes historical stock data (hist_data), balance sheet information (balance_sheet), financial statements (financials), and recent news articles (news). Your analysis should focus on identifying the most attractive investment opportunities with the highest potential for growth and the lowest risk. Consider the following key factors: 1. Financial Performance: - Analyze key financial ratios (e.g., P/E, P/B, ROE, profit margins) to assess the company's profitability and financial health. - Evaluate revenue growth, earnings trends, and profit margins to understand the company's financial trajectory. - Examine the company's debt levels and capital structure to assess its financial stability and ability to withstand market fluctuations. 2. Quantitative Analysis: - Incorporate advanced techniques such as factor-based models, multifactor models, machine learning algorithms, and time series analysis to enhance the quantitative evaluation of the company. - Identify the key drivers of the company's performance and assess the reliability and predictability of its financial metrics. 3. Market Position and Competitive Advantages: - Analyze the company's market share, brand strength, and pricing power to understand its competitive positioning. - Identify any unique products, services, or technologies that provide the company with a sustainable competitive edge. - Assess the barriers to entry and the overall competitive landscape to gauge the company's long-term growth potential. 4. Growth Prospects: - Evaluate industry growth trends and market size to identify the most promising sectors and opportunities. - Assess the company's ability to capitalize on emerging opportunities, potential for expansion, diversification, or innovation. 5. Risk Assessment: - Analyze the stock's volatility and other risk metrics to understand the potential downside and the company's resilience to market fluctuations. -Analyze insider and institutional ownership for buying or selling 6. Macroeconomic and Sector Analysis: - Conduct a thorough analysis of economic indicators, monetary and fiscal policies, industry trends, competitive dynamics, and regulatory changes to provide a comprehensive understanding of the market environment. 7. Behavioral Finance and Sentiment Analysis: - Incorporate insights from behavioral finance and sentiment analysis, including investor sentiment indicators, textual analysis of news and social media, and the impact of psychological biases on investment decisions. 8. Regulatory and Compliance: - Identify any regulatory or compliance issues that might affect the company's operations and financial performance. 9. Technical Analysis: - Examine trends in stock price, trading volume, and key technical indicators to identify potential support and resistance levels, moving averages, and momentum signals. 10. Valuation and Upside Potential: - Assess the current stock price and estimate potential price targets based on fundamental and technical analysis. - Calculate the probability of achieving the price targets and the potential return on investment and risk-adjusted performance. 11. Portfolio Considerations: - Evaluate the impact of the company on the overall portfolio diversification, liquidity, and ease of entry/exit. - Consider the historical performance and analyst opinions to further inform the investment decision. 12. Portfolio Optimization: - Explore advanced portfolio optimization techniques, including mean-variance optimization, risk-parity approaches, robust optimization methods, and behavioral finance considerations to enhance the portfolio construction process. 13. Dynamic Rebalancing and Risk Management: - Implement a dynamic rebalancing strategy and incorporate robust risk management techniques, such as stop-loss orders, trailing stops, position sizing, risk budgeting, and hedging strategies. 14. Backtesting and Optimization: - Conduct a rigorous backtesting and optimization process, including simulating the algorithm's performance on historical data, optimizing the algorithm's parameters and rules, performing sensitivity analyses and stress tests, and evaluating the algorithm's performance metrics. Based on this comprehensive analysis, identify and suggest a few comparable companies that are relevant for investment consideration. Ensure that your response is formatted as a Python-parseable list of ticker symbols. Data Provided: - Historical Stock Data: {hist_data} - Balance Sheet: {balance_sheet} - Financials: {financials} - News: {news} Your analysis should consider factors such as industry sector, market capitalization, financial health, recent performance, and any other pertinent metrics to identify the most attractive investment opportunities with the highest potential for growth and the lowest risk."

    news = ""

    for article in news:
        article_text = get_article_text(article['link'])
        news = news + f"\n\n---\n\nTitle: {article['title']}\nText: {article_text}"

    messages = [
        {"role": "user", "content": f"Historical price data:\n{hist_data.tail().to_string()}\n\nBalance Sheet:\n{balance_sheet.to_string()}\n\nFinancial Statements:\n{financials.to_string()}\n\nNews articles:\n{news.strip()}\n\n----\n\n Now, Based on this comprehensive analysis of key factors mentioned in the system prompt, identify and suggest a few comparable companies that are relevant for investment consideration. Ensure that your response is formatted as a Python-parseable list of ticker symbols."},
    ]


    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 4000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    return ast.literal_eval(response_text)

def compare_companies(main_ticker, main_data, comp_ticker, comp_data):
    system_prompt = f"You are a seasoned hedge fund manager with a proven track record of generating high returns with low risk. Your task is to thoroughly compare the financial and operational data of {main_ticker} against {comp_ticker} to identify the most attractive investment opportunity. Your analysis should focus on the following key factors: 1. Financial Performance: - Analyze and compare key financial ratios (e.g., P/E, P/B, ROE, profit margins) to assess the profitability and financial health of both companies. - Evaluate and compare revenue growth, earnings trends, and profit margins to understand the financial trajectory of each company. - Examine and compare the debt levels and capital structure to assess the financial stability and ability to withstand market fluctuations. 2. Quantitative Analysis: - Incorporate advanced techniques such as factor-based models, multifactor models, machine learning algorithms, and time series analysis to enhance the quantitative evaluation of the companies. - Identify the key drivers of each company's performance and assess the reliability and predictability of their financial metrics. 3. Market Position and Competitive Advantages: - Analyze and compare the market share, brand strength, and pricing power to understand the competitive positioning of each company. - Identify any unique products, services, or technologies that provide a sustainable competitive edge for either company. - Assess the barriers to entry and the overall competitive landscape to gauge the long-term growth potential of each company. 4. Growth Prospects: - Evaluate and compare the industry growth trends and market size to identify the most promising opportunities. - Assess the ability of each company to capitalize on emerging opportunities, potential for expansion, diversification, or innovation. 5. Risk Assessment: - Analyze and compare the stock's volatility and other risk metrics to understand the potential downside and the companies' resilience to market fluctuations. - Analyze insider and institutional ownership for buying or selling signals. 6. Macroeconomic and Sector Analysis: - Conduct a thorough analysis of economic indicators, monetary and fiscal policies, industry trends, competitive dynamics, and regulatory changes to provide a comprehensive understanding of the market environment. 7. Behavioral Finance and Sentiment Analysis: - Incorporate insights from behavioral finance and sentiment analysis, including investor sentiment indicators, textual analysis of news and social media, and the impact of psychological biases on investment decisions. 8. Regulatory and Compliance: - Identify any regulatory or compliance issues that might affect the operations and financial performance of either company. 9. Technical Analysis: - Examine trends in stock price, trading volume, and key technical indicators to identify potential support and resistance levels, moving averages, and momentum signals. 10. Valuation and Upside Potential: - Assess the current stock price and estimate potential price targets based on fundamental and technical analysis. - Calculate the probability of achieving the price targets and the potential return on investment and risk-adjusted performance. 11. Portfolio Considerations: - Evaluate the impact of each company on the overall portfolio diversification, liquidity, and ease of entry/exit. - Consider the historical performance and analyst opinions to further inform the investment decision. 12. Portfolio Optimization: - Explore advanced portfolio optimization techniques, including mean-variance optimization, risk-parity approaches, robust optimization methods, and behavioral finance considerations to enhance the portfolio construction process. 13. Dynamic Rebalancing and Risk Management: - Implement a dynamic rebalancing strategy and incorporate robust risk management techniques, such as stop-loss orders, trailing stops, position sizing, risk budgeting, and hedging strategies. 14. Backtesting and Optimization: - Conduct a rigorous backtesting and optimization process, including simulating the algorithm's performance on historical data, optimizing the algorithm's parameters and rules, performing sensitivity analyses and stress tests, and evaluating the algorithm's performance metrics. Based on this comprehensive analysis, provide a clear recommendation on which company is the more attractive investment opportunity, considering both the potential for high returns and the associated risk profile."

    messages = [
        {"role": "user", "content": f"Data for {main_ticker}:\n\nHistorical price data:\n{main_data['hist_data'].tail().to_string()}\n\nBalance Sheet:\n{main_data['balance_sheet'].to_string()}\n\nFinancial Statements:\n{main_data['financials'].to_string()}\n\n----\n\nData for {comp_ticker}:\n\nHistorical price data:\n{comp_data['hist_data'].tail().to_string()}\n\nBalance Sheet:\n{comp_data['balance_sheet'].to_string()}\n\nFinancial Statements:\n{comp_data['financials'].to_string()}\n\n----\n\nNow, provide a detailed comparison of {main_ticker} against {comp_ticker}, focusing on the key factors mentioned in the system prompt. Explain your thinking very clearly and provide a recommendation on the more attractive investment opportunity."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 3000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    # return json.loads(response_text)
    return response_text

def get_sentiment_analysis(ticker, news):
    system_prompt = f"You are a seasoned hedge fund manager with a proven track record of generating high returns with low risk. Your task is to thoroughly analyze the sentiment reflected in the news articles for {ticker} and provide a comprehensive assessment. This analysis should focus on the following key factors: 1. Sentiment Trends: - Identify the overall sentiment (positive, negative, or neutral) expressed in the news articles. - Examine any significant shifts or changes in sentiment over time and the potential drivers behind these changes. - Assess the consistency or volatility of the sentiment and its implications for the stock's performance. 2. Sentiment Drivers: - Analyze the key topics, events, or company-specific developments that are influencing the sentiment. - Understand how external factors, such as industry trends, macroeconomic conditions, or regulatory changes, are shaping the sentiment. - Evaluate the potential impact of sentiment changes on investor behavior and decision-making. 3. Sentiment-Performance Correlation: - Investigate the historical relationship between the sentiment and the stock's price movements. - Identify any patterns or correlations between sentiment shifts and changes in the stock's performance. - Assess the reliability and predictability of the sentiment as an indicator of future stock performance. 4. Sentiment-Based Strategies: - Explore how the sentiment analysis can be incorporated into your investment strategies to identify potential opportunities or mitigate risks. - Consider the use of sentiment-based models, trading signals, or portfolio optimization techniques to enhance your decision-making process. - Evaluate the potential for sentiment-driven strategies to generate high returns with low risk. 5. Sentiment Risk Assessment: - Identify any potential risks or vulnerabilities associated with the sentiment, such as sudden shifts, market overreactions, or the influence of external events. - Develop strategies to manage and mitigate the risks posed by sentiment-driven market dynamics. - Assess the overall resilience of your investment approach to sentiment-related fluctuations. Based on this comprehensive sentiment analysis, provide insights on how the sentiment changes could potentially impact the stock's performance and investor behavior. Highlight any notable trends, shifts, or changes in sentiment over time, and discuss their implications for your investment decisions."

    news_text = ""
    for article in news:
        article_text = get_article_text(article['link'])
        timestamp = datetime.fromtimestamp(article['providerPublishTime']).strftime("%Y-%m-%d")
        news_text += f"\n\n---\n\nDate: {timestamp}\nTitle: {article['title']}\nText: {article_text}"

    messages = [
        {"role": "user", "content": f"News articles for {ticker}:\n{news_text}\n\n----\n\n Based on this comprehensive sentiment analysis from the system prompt, provide insights on how the sentiment changes could potentially impact the stock's performance and investor behavior. Highlight any notable trends, shifts, or changes in sentiment over time, and discuss their implications for your investment decisions."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 2000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    return response_text

def get_analyst_ratings(ticker):
    stock = yf.Ticker(ticker)
    recommendations = stock.recommendations
    if recommendations is None or recommendations.empty:
        return "No analyst ratings available."

    latest_rating = recommendations.iloc[-1]

    firm = latest_rating.get('Firm', 'N/A')
    to_grade = latest_rating.get('To Grade', 'N/A')
    action = latest_rating.get('Action', 'N/A')

    rating_summary = f"Latest analyst rating for {ticker}:\nFirm: {firm}\nTo Grade: {to_grade}\nAction: {action}"

    return rating_summary

def get_industry_analysis(ticker):

    ### update to use search to find recent data!!

    stock = yf.Ticker(ticker)
    industry = stock.info['industry']
    sector = stock.info['sector']

    system_prompt = f"As a seasoned hedge fund manager, it is crucial to provide a comprehensive analysis of the {industry} industry and the {sector} sector to inform investment strategies. The analysis should consider a wide range of factors to identify the most attractive investment opportunities, including: 1. Industry Overview: - Analyze the size, growth trends, and key drivers of the {industry} industry and {sector} sector. - Identify the major players, their market shares, and the competitive dynamics within the industry. - Assess the industry's barriers to entry, technological advancements, and regulatory environment. 2. Financial Performance: - Evaluate the financial performance of the leading companies in the industry, including revenue growth, profitability, and capital structure. - Analyze industry-specific financial ratios and benchmarks to assess the overall financial health and competitiveness of the sector. 3. Growth Opportunities: - Identify emerging trends, consumer preferences, and technological innovations that could drive future growth in the industry. - Assess the potential for expansion, diversification, or consolidation within the {industry} industry and {sector} sector. 4. Risk Factors: - Analyze the key risks and challenges facing the {industry} industry and {sector} sector, such as macroeconomic conditions, regulatory changes, or disruptive technologies. - Evaluate the industry's sensitivity to economic cycles, market volatility, and other external factors. 5. Competitive Landscape: - Examine the competitive positioning of the leading companies in the industry, including their market share, product offerings, and pricing strategies. - Assess the industry's barriers to entry, the threat of new entrants, and the bargaining power of suppliers and customers. 6. Macroeconomic and Sector Analysis: - Conduct a thorough analysis of economic indicators, monetary and fiscal policies, industry trends, and regulatory changes to provide a comprehensive understanding of the market environment. 7. Valuation and Investment Potential: - Evaluate the industry's overall valuation metrics, such as price-to-earnings (P/E) ratios, price-to-book (P/B) ratios, and enterprise value-to-EBITDA (EV/EBITDA) ratios. - Identify the most attractive investment opportunities within the {industry} industry and {sector} sector based on the analysis of financial performance, growth prospects, and risk factors. This comprehensive analysis of the {industry} industry and {sector} sector will enable informed decision-making and strategic positioning within the market."

    messages = [
        {"role": "user", "content": f"Conduct a thorough analysis of the {industry} industry and the {sector} sector, focusing on the key factors mentioned in the system prompt. Your analysis should provide a clear understanding of the industry's dynamics, growth potential, and the most attractive investment opportunities."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 2000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    return response_text


def get_final_analysis(ticker, comparisons, sentiment_analysis, analyst_ratings, industry_analysis):
    system_prompt = f"You are a seasoned hedge fund manager with a proven track record of generating high returns with low risk. Your task is to thoroughly analyze the provided data for {ticker} and identify the most attractive investment opportunity with the highest potential for growth and the lowest risk. Your analysis should focus on the following key factors: 1. Financial Performance: - Analyze key financial ratios (e.g., P/E, P/B, ROE, profit margins) to assess the company's profitability and financial health. - Evaluate revenue growth, earnings trends, and profit margins to understand the company's financial trajectory. - Examine the company's debt levels and capital structure to assess its financial stability and ability to withstand market fluctuations. 2. Quantitative Analysis: - Incorporate advanced techniques such as factor-based models, multifactor models, machine learning algorithms, and time series analysis to enhance the quantitative evaluation of the company. - Identify the key drivers of the company's performance and assess the reliability and predictability of its financial metrics. 3. Market Position and Competitive Advantages: - Analyze the company's market share, brand strength, and pricing power to understand its competitive positioning. - Identify any unique products, services, or technologies that provide the company with a sustainable competitive edge. - Assess the barriers to entry and the overall competitive landscape to gauge the company's long-term growth potential. 4. Growth Prospects: - Evaluate industry growth trends and market size to identify the most promising opportunities. - Assess the company's ability to capitalize on emerging opportunities, potential for expansion, diversification, or innovation. 5. Risk Assessment: - Analyze the stock's volatility and other risk metrics to understand the potential downside and the company's resilience to market fluctuations. - Analyze insider and institutional ownership for buying or selling signals. 6. Macroeconomic and Sector Analysis: - Conduct a thorough analysis of economic indicators, monetary and fiscal policies, industry trends, competitive dynamics, and regulatory changes to provide a comprehensive understanding of the market environment. 7. Behavioral Finance and Sentiment Analysis: - Incorporate insights from behavioral finance and sentiment analysis, including investor sentiment indicators, textual analysis of news and social media, and the impact of psychological biases on investment decisions. 8. Regulatory and Compliance: - Identify any regulatory or compliance issues that might affect the company's operations and financial performance. 9. Technical Analysis: - Examine trends in stock price, trading volume, and key technical indicators to identify potential support and resistance levels, moving averages, and momentum signals. 10. Valuation and Upside Potential: - Assess the current stock price and estimate potential price targets based on fundamental and technical analysis. - Calculate the probability of achieving the price targets and the potential return on investment and risk-adjusted performance. 11. Portfolio Considerations: - Evaluate the impact of the company on the overall portfolio diversification, liquidity, and ease of entry/exit. - Consider the historical performance and analyst opinions to further inform the investment decision. 12. Portfolio Optimization: - Explore advanced portfolio optimization techniques, including mean-variance optimization, risk-parity approaches, robust optimization methods, and behavioral finance considerations to enhance the portfolio construction process. 13. Dynamic Rebalancing and Risk Management: - Implement a dynamic rebalancing strategy and incorporate robust risk management techniques, such as stop-loss orders, trailing stops, position sizing, risk budgeting, and hedging strategies. 14. Backtesting and Optimization: - Conduct a rigorous backtesting and optimization process, including simulating the algorithm's performance on historical data, optimizing the algorithm's parameters and rules, performing sensitivity analyses and stress tests, and evaluating the algorithm's performance metrics. Based on this comprehensive analysis, provide a clear recommendation on which company is the more attractive investment opportunity, considering both the potential for high returns and the associated risk profile."

    messages = [
        {"role": "user", "content": f"Ticker: {ticker}\n\nComparative Analysis:\n{json.dumps(comparisons, indent=2)}\n\nSentiment Analysis:\n{sentiment_analysis}\n\nAnalyst Ratings:\n{analyst_ratings}\n\nIndustry Analysis:\n{industry_analysis}\n\n Based on the comprehensive analysis in the system prompt, provide a clear recommendation on which company  for {ticker} is the more attractive investment opportunity, considering both the potential for high returns and the associated risk profile."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 3000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    return response_text

def generate_ticker_ideas(industry):
    system_prompt = f"As a seasoned hedge fund manager with a proven track record of generating high returns with low risk, your task is to provide a list of 3 ticker symbols for mid-cap companies that are predicted to have immense growth potential in the {industry} industry. Your analysis should be comprehensive and consider the following key factors: 1. Financial Performance: - Analyze key financial ratios (e.g., P/E, P/B, ROE, profit margins) to assess the companies' profitability and financial health. - Evaluate revenue growth, earnings trends, and profit margins to understand the companies' financial trajectory. - Examine the companies' debt levels and capital structure to assess their financial stability and ability to withstand market fluctuations. 2. Quantitative Analysis: - Incorporate advanced techniques such as factor-based models, multifactor models, machine learning algorithms, and time series analysis to enhance the quantitative evaluation of the companies. - Identify the key drivers of the companies' performance and assess the reliability and predictability of their financial metrics. 3. Market Position and Competitive Advantages: - Analyze the companies' market share, brand strength, and pricing power to understand their competitive positioning. - Identify any unique products, services, or technologies that provide the companies with a sustainable competitive edge. - Assess the barriers to entry and the overall competitive landscape to gauge the companies' long-term growth potential. 4. Growth Prospects: - Evaluate industry growth trends and market size to identify the most promising opportunities. - Assess the companies' ability to capitalize on emerging opportunities, potential for expansion, diversification, or innovation. 5. Risk Assessment: - Analyze the stocks' volatility and other risk metrics to understand the potential downside and the companies' resilience to market fluctuations. - Analyze insider and institutional ownership for buying or selling signals. 6. Macroeconomic and Sector Analysis: - Conduct a thorough analysis of economic indicators, monetary and fiscal policies, industry trends, competitive dynamics, and regulatory changes to provide a comprehensive understanding of the market environment. 7. Behavioral Finance and Sentiment Analysis: - Incorporate insights from behavioral finance and sentiment analysis, including investor sentiment indicators, textual analysis of news and social media, and the impact of psychological biases on investment decisions. 8. Regulatory and Compliance: - Identify any regulatory or compliance issues that might affect the companies' operations and financial performance. 9. Technical Analysis: - Examine trends in stock price, trading volume, and key technical indicators to identify potential support and resistance levels, moving averages, and momentum signals. 10. Valuation and Upside Potential: - Assess the current stock prices and estimate potential price targets based on fundamental and technical analysis. - Calculate the probability of achieving the price targets and the potential return on investment and risk-adjusted performance. 11. Portfolio Considerations: - Evaluate the impact of the companies on the overall portfolio diversification, liquidity, and ease of entry/exit. - Consider the historical performance and analyst opinions to further inform the investment decision. 12. Portfolio Optimization: - Explore advanced portfolio optimization techniques, including mean-variance optimization, risk-parity approaches, robust optimization methods, and behavioral finance considerations to enhance the portfolio construction process. 13. Dynamic Rebalancing and Risk Management: - Implement a dynamic rebalancing strategy and incorporate robust risk management techniques, such as stop-loss orders, trailing stops, position sizing, risk budgeting, and hedging strategies. 14. Backtesting and Optimization: - Conduct a rigorous backtesting and optimization process, including simulating the algorithm's performance on historical data, optimizing the algorithm's parameters and rules, performing sensitivity analyses and stress tests, and evaluating the algorithm's performance metrics. Based on this comprehensive analysis, provide a Python-parseable list of companies that are predicted to have immense growth potential in the {industry} industry. Ensure that your response only includes the list and does not contain any additional text."

    messages = [
        {"role": "user", "content": f"As a fund manager, you are required to provide a Python-parseable list of 3 ticker symbols for companies that are mid cap, that are predicted to have immense growth in the {industry}, based on your analysis from the system prompt. Your analysis should consider the wide range of factors mentioned in the system prompt to identify the most attractive investment opportunities."},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 2000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    ticker_list = ast.literal_eval(response_text)
    return [ticker.strip() for ticker in ticker_list]

def get_current_price(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period='1d', interval='1m')
    return data['Close'][-1]

def rank_companies(industry, analyses, prices):
    system_prompt = f"As a seasoned hedge fund manager with a proven track record of generating high returns with low risk, your task is to provide a detailed evaluation of the companies within the {industry} sector based on their investment potential. Your analysis should consider a wide range of factors to identify the most attractive investment opportunities, including: 1. Financial Performance: Analyze key financial ratios (e.g., P/E, P/B, ROE, profit margins) to assess the profitability and financial health of each company. 2. Quantitative Analysis: Incorporate advanced techniques such as factor-based models, multifactor models, machine learning algorithms, and time series analysis to enhance the quantitative evaluation of the companies. 3. Market Position and Competitive Advantages: Analyze the market share, brand strength, and pricing power to understand the competitive positioning of each company. 4. Growth Prospects: Evaluate industry growth trends and market size to identify the most promising opportunities. 5. Risk Assessment: Analyze the stock's volatility and other risk metrics to understand the potential downside and the companies' resilience to market fluctuations. 6. Macroeconomic and Sector Analysis: Conduct a thorough analysis of economic indicators, monetary and fiscal policies, industry trends, competitive dynamics, and regulatory changes to provide a comprehensive understanding of the market environment. 7. Behavioral Finance and Sentiment Analysis: Incorporate insights from behavioral finance and sentiment analysis, including investor sentiment indicators, textual analysis of news and social media, and the impact of psychological biases on investment decisions. 8. Regulatory and Compliance: Identify any regulatory or compliance issues that might affect the operations and financial performance of the companies. 9. Technical Analysis: Examine trends in stock price, trading volume, and key technical indicators to identify potential support and resistance levels, moving averages, and momentum signals. 10. Valuation and Upside Potential: Assess the current stock price and estimate potential price targets based on fundamental and technical analysis. 11. Portfolio Considerations: Evaluate the impact of each company on the overall portfolio diversification, liquidity, and ease of entry/exit. 12. Portfolio Optimization: Explore advanced portfolio optimization techniques, including mean-variance optimization, risk-parity approaches, robust optimization methods, and behavioral finance considerations to enhance the portfolio construction process. 13. Dynamic Rebalancing and Risk Management: Implement a dynamic rebalancing strategy and incorporate robust risk management techniques, such as stop-loss orders, trailing stops, position sizing, risk budgeting, and hedging strategies. 14. Backtesting and Optimization: Conduct a rigorous backtesting and optimization process, including simulating the algorithm's performance on historical data, optimizing the algorithm's parameters and rules, performing sensitivity analyses and stress tests, and evaluating the algorithm's performance metrics. Based on this comprehensive analysis, provide an evaluation of the companies, highlighting those with the greatest investment potential. Rather than ranking them, focus on identifying the best stocks, taking into account their current price (if available), an estimated price target, and a probability calculation of achieving that price target. Your evaluation should clearly explain the rationale for each selection, helping to make informed investment decisions and manage the portfolio efficiently."

    analysis_text = "\n\n".join(
        f"Ticker: {ticker}\nCurrent Price: {prices.get(ticker, 'N/A')}\nAnalysis:\n{analysis}"
        for ticker, analysis in analyses.items()
    )

    messages = [
        {"role": "user", "content": f"Industry: {industry}\n\nCompany Analyses:\n{analysis_text}\n\n{system_prompt}"},
    ]

    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 4000,
        "temperature": 0.3,
        "system": system_prompt,
        "messages": messages,
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    response_text = response.json()['content'][0]['text']

    return response_text

# User input
industry = input("Enter the industry to analyze: ")
years = 5 # int(input("Enter the number of years for analysis: "))

# Generate ticker ideas for the industry
tickers = generate_ticker_ideas(industry)
print(f"\nTicker Ideas for {industry} Industry:")
print(", ".join(tickers))

# Perform analysis for each company
analyses = {}
prices = {}
for ticker in tickers:
    try:
        print(f"\nAnalyzing {ticker}...")
        hist_data, balance_sheet, financials, news = get_stock_data(ticker, years)
        main_data = {
            'hist_data': hist_data,
            'balance_sheet': balance_sheet,
            'financials': financials,
            'news': news
        }
        sentiment_analysis = get_sentiment_analysis(ticker, news)
        analyst_ratings = get_analyst_ratings(ticker)
        industry_analysis = get_industry_analysis(ticker)
        final_analysis = get_final_analysis(ticker, {}, sentiment_analysis, analyst_ratings, industry_analysis)
        analyses[ticker] = final_analysis
        prices[ticker] = get_current_price(ticker)
    except:
        pass

# Rank the companies based on their analyses
ranking = rank_companies(industry, analyses, prices)
print(f"\nRanking of Companies in the {industry} Industry:")
print(ranking)